In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!ls

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

#supress warnings
import warnings
warnings.filterwarnings("ignore")
import os
import statistics

In [4]:
directories = ['../input/csc4851-homework4/birds_400/test',
                                '../input/csc4851-homework4/birds_400/train',
                                '../input//csc4851-homework4/birds_400/valid']

for dir in directories:
    label = []
    path = []
    for dirname, _,filenames in os.walk(dir):
        for filename in filenames:
            label.append(os.path.split(dirname)[1])
            path.append(os.path.join(dirname,filename))
    if dir == directories[0]:
        df_test = pd.DataFrame(columns=['path','label'])
        df_test['path']=path
        df_test['label']=label
    elif dir == directories[1]:
        df_train = pd.DataFrame(columns=['path','label'])
        df_train['path']=path
        df_train['label']=label        
    elif dir == directories[2]:
        df_valid = pd.DataFrame(columns=['path','label'])
        df_valid['path']=path
        df_valid['label']=label


In [5]:
df_train.head()

In [6]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms

from torchvision.datasets import ImageFolder
from torch.autograd import Variable
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader

from PIL import Image
import torch.nn.functional as F

In [7]:
# Hyper Parameters 
input_size = 4320000         # this is 224x224 I think this is mainly for logistic regression
num_classes = 400            # Length of bird in csv file
num_epochs = 200              # this is fine may need to increase it 
batch_size = 256             # tune batch size according to CPU or the GPU
learning_rate = 0.001

device = "cuda" if torch.cuda.is_available() else "cpu" 

In [8]:
device

In [9]:
trans = transforms.Compose([                                           # Defining a variable transforms
                    transforms.Resize((150,150)),                      # Resize the image to 256x256 pixels
                    #transforms.CenterCrop(224),                       # Crop the image to 224x224 pixels about the center
                    transforms.ToTensor(),                             # 0-255 to 0-1 numpy to tensor
                    transforms.Normalize(mean=(0.5, 0.5, 0.5),         # Normalize the image
                                        std= (0.5, 0.5, 0.5))           # Mean and std of image as also used when training
                    ])

In [10]:
# PyTorch DataLoader
directories = ['../input/csc4851-homework4/birds_400/test',
                                '../input/csc4851-homework4/birds_400/train',
                                '../input//csc4851-homework4/birds_400/valid']
train_loader = DataLoader(ImageFolder(directories[0], transform=trans), 
                          batch_size=64, shuffle=True, #num_workers=3, pin_memory=True
                        )
test_loader  = DataLoader(ImageFolder(directories[1], transform=trans), 
                          batch_size=32, shuffle=True, #num_workers=3, pin_memory=True
                        )
valid_loader = DataLoader(ImageFolder(directories[2], transform=trans), 
                          batch_size, #num_workers=3, pin_memory=True
                        )

In [11]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=400):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape = (256,12,150,150)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1 = nn.ReLU()
        #Shape = (256,12,150,150)
        
        self.pool = nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape = (256,12,75,75)
        
        
        self.conv2 = nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2 = nn.ReLU()
        #Shape = (256,20,75,75)
        
        
        self.conv3 = nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3 = nn.ReLU()
        #Shape= (256,32,75,75)
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
    #Feed forwad function    
    def forward(self,inp):
        output=self.conv1(inp)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
        #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
        output=self.fc(output)
            
        return output


In [12]:
class AlexNet(torch.nn.Module):

    def __init__(self, num_classes):
        super().__init__()
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(64, 192, kernel_size=5, padding=2),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
            torch.nn.Conv2d(192, 384, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(384, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(inplace=True),
            torch.nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = torch.nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(0.5),
            torch.nn.Linear(256 * 6 * 6, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        logits = self.classifier(x)
        return logits

In [13]:
import glob
import pathlib

root = pathlib.Path(directories[0])
classes = sorted(j.name.split('/')[-1] for j in root.iterdir())             

#model = ConvNet(num_classes).to(device)
model = AlexNet(num_classes).to(device)

# Optimizer and loss function
# Parameters have been set as global values
loss_function = nn.CrossEntropyLoss()  
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.0001)  
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

train_count = len(glob.glob(directories[0]+'/**/*.jpg'))
test_count = len(glob.glob(directories[1]+'/**/*.jpg'))


In [14]:
print(classes)
print(train_count)
print(test_count)

In [15]:
def train(model, loss_fn, optimizer):
    
    #set the module in training mode
    model.train()
    
    train_batch_losses = []
    
    for batch, labels in train_loader:
        
        #send the training data to the GPU
        batch = batch.to(device)
        labels = labels.to(device)
        
        #set all gradients to zero
        optimizer.zero_grad()
        
        #forward propagate
        y_pred = model(batch)
        
        #calculate the loss
        loss = loss_fn(y_pred, labels)
        
        #bachpropagate
        loss.backward()
        
        #update the parameters (weights and biases)
        optimizer.step()
        
        train_batch_losses.append(float(loss))
        
    mean_loss = statistics.mean(train_batch_losses)
        
    return mean_loss

In [16]:
def validate(model, loss_fn, optimizer):
    
    # set the model in evaluation mode
    model.eval()
    
    # save predictions for later
    pedrictions = []
    
    # stop tracking the parameters for backpropagation
    with torch.no_grad():
        
        validation_batch_losses = []
        
        for batch, labels in valid_loader:
            
            # send the validation data to GPU
            batch = batch.to(device)
            labels = labels.to(device)
            
            # forward propagate
            labels_pred = model(batch)
            
            # calculate loss
            loss = loss_fn(labels_pred, labels)
            
            validation_batch_losses.append(float(loss))
            
            mean_loss = statistics.mean(validation_batch_losses)
           
    return mean_loss 

In [17]:
def accuracy(model, loader):
    correct = 0
    total = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch, labels in loader:
            batch = batch.to(device)
            labels = labels.to(device)
            
            labels_pred = model(batch)
            
            _, predicted = torch.max(labels_pred.data, 1)
        
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            return (100 * correct / total)

In [18]:
def training_stats(train_loss, train_accuracy, val_loss, val_accuracy):
    print(('training loss: {:.3f} '
           'training accuracy: {:.2f}% || '
           'val. loss: {:.3f} '
           'val. accuracy: {:.2f}%').format(train_loss, train_accuracy,
                                            val_loss, val_accuracy))

In [19]:
loss_fn = nn.CrossEntropyLoss()

train_losses = []
valid_losses = []

for epoch in range(1, 1+10):
    
    print('Epoch number', epoch)
    
    train_loss = train(model, loss_fn, optimizer)
    train_losses.append(train_loss)
    train_accuracy = accuracy(model, train_loader)
    
    valid_loss = validate(model, loss_fn, optimizer)
    valid_losses.append(valid_loss)
    valid_accuracy = accuracy(model, valid_loader)

    training_stats(train_loss, train_accuracy, valid_loss, valid_accuracy)

In [9]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation,BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from keras.preprocessing import image
from keras import losses
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import os
from sklearn.metrics import log_loss
from glob import glob
import csv  

In [10]:
#Parsing all the t
path = '/kaggle/input/csc4851-homework4/birds_400/train/'
train_datagen = ImageDataGenerator(rescale=1. / 224)
train = train_datagen.flow_from_directory(path, target_size=(224,224), class_mode='categorical')

classes = [x[0].split('/')[-1] for x in os.walk(path)]
classes.pop(0)

In [11]:
# Using alexnet
def AlexNet(input_shape):
    
    X_input = Input(input_shape)
    
    X = Conv2D(96,(11,11),strides = 4,name="conv0")(X_input)
    X = BatchNormalization(axis = 3 , name = "bn0")(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max0')(X)
    
    X = Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(X)
    X = BatchNormalization(axis = 3 ,name='bn1')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max1')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(256, (3,3) , padding = 'same' , name='conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max2')(X)
    
    X = Flatten()(X)
    
    X = Dense(4096, activation = 'relu', name = "fc0")(X)
    
    X = Dense(4096, activation = 'relu', name = 'fc1')(X) 
    
    X = Dense(400,activation='softmax',name = 'fc2')(X)
    model = Model(inputs = X_input, outputs = X, name='AlexNet')
    
    return model

In [12]:
alex = AlexNet(train[0][0].shape[1:])

In [13]:
alex.summary()

In [14]:
alex.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

In [15]:
alex.fit_generator(train,epochs=7)

In [ ]:
path_test = '/kaggle/input/csc4851-homework4/birds_400/test'
test_datagen = ImageDataGenerator(rescale=1. / 224)
test = test_datagen.flow_from_directory(path_test, target_size=(224,224), class_mode='categorical')

In [ ]:
preds = alex.evaluate_generator(test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]*100))

In [ ]:
#validation

path_test = '/kaggle/input/csc4851-homework4/birds_400/valid'
validation_datagen = ImageDataGenerator(rescale=1. / 224)
validation = validation_datagen.flow_from_directory(path_test, target_size=(224,224), batch_size = 1,class_mode='categorical')

In [ ]:
def prepare(img_path):
    img = image.load_img(img_path, target_size=(224,224))
    x = image.img_to_array(img)
    x = x/224
    return np.expand_dims(x, axis=0)

In [ ]:
cce = losses.CategoricalCrossentropy()

In [ ]:
# result = alex.predict([prepare("/kaggle/input/csc4851-homework4/birds_400/test/ABBOTTS BABBLER/1.jpg")])
# print(len(result[0]))
template = [0]*400
log_loss = {}
test_images = glob("/kaggle/input/csc4851-homework4/birds_400/test/" + "*/*.jpg")

for path in test_images:
    result = alex.predict([prepare(path)])
    actual_class = path.split('/')[-2]
    actual_class = "BLACK & YELLOW  BROADBILL" if actual_class == "BLACK & YELLOW BROADBILL" else actual_class
    actual_index = classes.index(actual_class)
    template[actual_index] = 1
    log_loss_current = cce(template, result[0]).numpy()
#     print(log_loss_current)
    if actual_index in log_loss:
        log_loss[actual_index] += (log_loss_current)/100
    else:
        log_loss[actual_index] = (log_loss_current)/100
    template[actual_index] = 0

# print(template)
# print(classes[int(x)])

In [ ]:
print('Generating submission.csv file...')
ids = list(log_loss.keys())
values = list(log_loss.values())

f = open('submission.csv', 'w')
writer = csv.writer(f)
writer.writerow(['id','birds'])
for index in range(len(ids)):
    writer.writerow([ids[index],values[index]])

!head submission.csv

In [ ]:
# Display 20 picture of the dataset with their labels
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(15, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

df_sample = df_train.sample(15)
df_sample.reset_index(drop=True, inplace=True)

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(df_sample.path[i]))
    ax.set_title(df_sample.label[i])
plt.tight_layout()
plt.show()